<a href="https://colab.research.google.com/github/jewelry715/likelion/blob/main/%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 별인지 펄서인지 확인하는 이진분류(sigmoid)

In [ ]:
import numpy as np
import csv
np.random.seed(1024)
rnd_mean=0
rnd_std=0.003
learning_rate=0.003

In [ ]:
%run /content/drive/MyDrive/abalone.ipynb

[[ 0.      1.      0.     ...  0.101   0.15   15.    ]
 [ 0.      1.      0.     ...  0.0485  0.07    7.    ]
 [ 0.      0.      1.     ...  0.1415  0.21    9.    ]
 ...
 [ 0.      1.      0.     ...  0.2875  0.308   9.    ]
 [ 0.      0.      1.     ...  0.261   0.296  10.    ]
 [ 0.      1.      0.     ...  0.3765  0.495  12.    ]]
[['M', '0.455', '0.365', '0.095', '0.514', '0.2245', '0.101', '0.15', '15'], ['M', '0.35', '0.265', '0.09', '0.2255', '0.0995', '0.0485', '0.07', '7']]
총 데이터의 수(행): 4177
데이터의 70%의 미니배치 스텝 수: 29


In [ ]:
def binary_classification_exec(epoch_count=1,mb_size=10,report=1,train_rate=0.8):
  binary_load_dataset()
  init_model()
  train_and_test(epoch_count,mb_size,report,train_rate)

In [ ]:
def binary_load_dataset():
  with open('/content/drive/MyDrive/pulsar_stars.csv') as csvfile:
    csvreader=csv.reader(csvfile)
    next(csvreader,None)
    rows=[]
    for row in csvreader:
      rows.append(row)

  global data,input_cnt,output_cnt
  input_cnt,output_cnt=8,1 #독립변수,종속변수
  data=np.asarray(rows,dtype='float32') #배열구조로 변환

In [ ]:
def init_model():
  global weight,bias,input_cnt,output_cnt
  weight=np.random.normal(rnd_mean,rnd_std,[input_cnt,output_cnt])
  bias=np.random.normal(rnd_mean,rnd_std,[output_cnt])

In [ ]:
def train_and_test(epoch_count, mb_size, report, train_rate):
  step_count = arrange_data(mb_size, train_rate)
  test_x, test_y = get_test_data()
  for epoch in range(epoch_count):
    losses, accs = [], []
    for n in range(step_count):
      train_x, train_y = get_train_data(mb_size, n)
      loss, acc = run_train(train_x, train_y)
      losses.append(loss)
      accs.append(acc)
    if report > 0 and (epoch+1) % report == 0:
      acc = run_test(test_x, test_y)
      print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
        format(epoch+1, np.mean(losses), np.mean(accs), acc))
  final_acc = run_test(test_x, test_y)
  print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

In [ ]:
def arrange_data(mb_size,train_rate):
  global data,shuffle_map,test_begin_idx #외부에서도 쓸 것
  shuffle_map=np.arange(data.shape[0])
  np.random.shuffle(shuffle_map)
  step_count=int(data.shape[0]*train_rate)//mb_size
  test_begin_idx=step_count*mb_size
  return step_count

In [ ]:
def get_test_data():
  global data,shuffle_map,test_begin_idx,output_cnt
  test_data=data[shuffle_map[test_begin_idx:]]
  return test_data[:,:-output_cnt],test_data[:,-output_cnt:]

In [ ]:
def get_train_data(mb_size,nth):
  global data,shuffle_map,test_begin_idx,output_cnt
  if nth==0:
    np.random.shuffle(shuffle_map[:test_begin_idx])
  train_data=data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
  return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [ ]:
def run_train(x,y):
  output,aux_nn=forward_neuralnet(x)#역전파 시 필요한 보조 지표
  loss,aux_pp=forward_postproc(output,y)
  accuracy=eval_accuracy(output,y)
  #순전파 과정
  g_loss=1.0
  g_output=backprop_postproc(g_loss,aux_pp)
  backprop_neuralnet(g_output,aux_nn)
  return loss,accuracy

In [ ]:
def run_test(x,y):
  output,_=forward_neuralnet(x)
  accuracy=eval_accuracy(output,y)
  return accuracy

In [ ]:
def backprop_neuralnet(g_output,x):
  global weight,bias
  g_output_w=x.transpose()
  g_w=np.matmul(g_output_w,g_output)
  weight-=learning_rate*g_w
  g_b=np.sum(g_output,axis=0)
  bias-=learning_rate*g_b

In [ ]:
def forward_postproc(output,y):#순전파 수행 함수
  cee=sigmoid_cross_entropy_with_logits(y,output)#로짓값 해석,시그모이드 함수,교차엔트로피를 거쳐 계산
  loss=np.mean(cee)#구해진 값들에 대해 평균을 낸다
  return loss,[y,output,cee] #역전파를 수행하기 위해 리스트 반환

In [ ]:
def backprop_postproc(g_loss,aux):
  y,output,cee=aux
  g_loss=1.0
  g_loss_entropy=1.0/np.prod(cee.shape) #교차 엔트로피를 변수화함
  g_entropy_output=sigmoid_cross_entropy_with_logits_derv(y,output) #교차 엔트로피에 대한 편미분
  g_entropy=g_loss_entropy*g_loss
  g_output=g_entropy_output*g_entropy #각 부분 기울기를 연쇄적으로 곱함
  return g_output

In [ ]:
def sigmoid(x):
  return np.exp(-relu(-x))/(1.0+np.exp(-np.abs(x))) #-max(x,0)->-relu(-x)
def sigmoid_derv(x,y):
  return y*(1-y)
def sigmoid_cross_entropy_with_logits(z,x):
  return relu(x)-x*z+np.log(1+np.exp(-np.abs(x)))
def sigmoid_cross_entropy_with_logits_derv(z,x):#x에 대해 편미분(시그모이드의 편미분 과정)
#연산과정 자체를 하나로 통합함
  return -z+sigmoid(x)
def relu(x):#입력값이 0보다 크면 그대로, 아니면 0으로 출력
  return np.maximum(x,0)

In [ ]:
def eval_accuracy(output,y):
  estimate=np.greater(output,0)#시그모이드 함수를 거친 판단
  answer=np.greater(y,0)#실제 정답으로 주어진 판단
  correct=np.equal(estimate,answer)#예측값과 실제가 일치할 경우ef eval_accuracy(output,y):
  return np.mean(correct)

In [ ]:
binary_classification_exec(epoch_count=10,mb_size=1,report=1,train_rate=0.8)

Epoch 1: loss=0.610, accuracy=0.952/0.969
Epoch 2: loss=0.613, accuracy=0.956/0.972
Epoch 3: loss=0.601, accuracy=0.958/0.346
Epoch 4: loss=0.633, accuracy=0.957/0.953
Epoch 5: loss=0.670, accuracy=0.959/0.963
Epoch 6: loss=0.560, accuracy=0.960/0.975
Epoch 7: loss=0.590, accuracy=0.959/0.976
Epoch 8: loss=0.587, accuracy=0.959/0.974
Epoch 9: loss=0.582, accuracy=0.960/0.872
Epoch 10: loss=0.538, accuracy=0.960/0.947

Final Test: final accuracy = 0.947


데이터셋의 균형이 무너져 정확도가 정확하게 보여지지 않는 상황을 타개하기 위해 신경망의 성능을 더욱 잘 보여줄 수 있는 평가지표가 필요함

=>정밀도,재현율

정밀도:신경망이 참으로 예측한 것 중에 정답이 참인 비율
- TP/(TP+FP)

재현율: 거꾸로 정답이 참인 것들 중에 신경망이 참으로 예측한 비율
- TP/(TP+FN)

- TP : 신경망의 추측이 '참(P)'이며, 데이터의 정답 또한 '참'(T)으로 정확하게 평가한 결과
- TN : 신경망의 추측이 '거짓(N)'이며, 데이터의 정답은 '참'(T)으로 부정확하게 평가한 결과
- FP : 신경망의 추측이 '참(P)'이며, 데이터의 정답은 '거짓(F)'으로 부정확하게 평가한 결과
- FN : 신경망의 추측이 '거짓(N)'이며, 데이터의 정답 또한 '거짓(F)'으로 정확하게 평가한 결과


F1: 정밀도와 재현율의 조화평균

F1=1/((1/a+a/b)/2)=2/(1/a+1/b)
=2/(1/정밀도+1/재현율)=2/((TP+FP)/TP+(TP+FN)/TP)
=2*TP/(2*TP+FP+FN)

정확성=(TP+TN)/(TP+FN+TN+FP)

In [ ]:
def binary_classification_exec(epoch_count=10, mb_size=10, report=1,train_rate = 0.8, adjust_ratio = False):
    binary_load_dataset(adjust_ratio)
    init_model()
    train_and_test(epoch_count, mb_size, report, train_rate)
  #PULSAR 데이터 증폭

In [ ]:
def binary_load_dataset(adjust_ratio):
  pulsars,stars=[],[]
  with open('/content/drive/MyDrive/pulsar_stars.csv') as csvfile:
    csvreader=csv.reader(csvfile)
    next(csvreader,None)
    
    for row in csvreader:
      if row[8] == '1' : pulsars.append(row)
      else:
        stars.append(row)

  global data,input_cnt,output_cnt
  input_cnt,output_cnt=8,1 #독립변수,종속변수
  star_cnt,pulsar_cnt=len(stars),len(pulsars)
  if adjust_ratio: #기존 데이터버퍼보다 크기를 2배 늘려줌
    data=np.zeros([2*star_cnt,9])
    data[0:star_cnt,:]=np.asarray(stars,dtype='float32') #배열구조로 변환

    for n in range(star_cnt):#절반은 star,절반은 pulsar
      data[star_cnt+n]=np.asarray(pulsars[n%pulsar_cnt],dtype='float32')
  else:
    data = np.zeros([star_cnt+pulsar_cnt,9])
    data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
    data[star_cnt:,:] = np.asarray(pulsars, dtype='float32')

In [ ]:
def eval_accuracy(output,y):
  est_yes=np.greater(output,0)
  ans_yes=np.greater(y,0.5)
  est_no=np.logical_not(est_yes)
  ans_no=np.logical_not(ans_yes)
  tp=np.sum(np.logical_and(est_yes,ans_yes))
  fp=np.sum(np.logical_and(est_yes,ans_no))
  fn=np.sum(np.logical_and(est_no,ans_no))
  tn=np.sum(np.logical_and(est_no,ans_yes))
  accuracy=safe_div(tp+tn,tp+fp+fn+tn)
  precision=safe_div(tp,tp+fp)
  recall=safe_div(tp,tp+fn)
  f1=2*safe_div(recall*precision,recall+precision)
  return [accuracy,precision,recall,f1]

In [ ]:
def safe_div(p,q):
  p,q=float(p),float(q)
  if np.abs(q)<1.0e-20:
    return np.sign(p)
  return p/q

In [ ]:
def train_and_test(epoch_count,mb_size,report,train_rate):
  step_count=arrange_data(mb_size,train_rate)
  test_x,test_y=get_test_data()
  for epoch in range(epoch_count):
    losses=[]
    for n in range(step_count):
      train_x,train_y=get_train_data(mb_size,n)
      loss,acc=run_train(train_x,train_y)
      losses.append(loss)
      
    if report>0 and (epoch+1)%report==0:
      acc=run_test(test_x,test_y)
      print("epoch{}:TRAIN -LOSS = {:5.3f},\n accuracy:{:5.3f},precision:{:5.3f},recall:{:5.3f},f1:{:5.3f}".\
                    format(epoch+1,np.mean(losses), acc[0],acc[1],acc[2],acc[3]))
  final_acc = run_test(test_x,test_y)
  print("\n 최종 테스트 :\n accuracy:{:5.3f},precision:{:5.3f},recall:{:5.3f},f1:{:5.3f}".\
          format(final_acc[0],final_acc[1],final_acc[2],final_acc[3]))


In [ ]:
binary_classification_exec(epoch_count=1000, report=100,mb_size = 10,adjust_ratio=False)

epoch100:TRAIN -LOSS = 0.128,
 accuracy:0.098,precision:0.915,recall:0.080,f1:0.147
epoch200:TRAIN -LOSS = 0.135,
 accuracy:0.098,precision:0.971,recall:0.069,f1:0.128
epoch300:TRAIN -LOSS = 0.126,
 accuracy:0.098,precision:0.920,recall:0.083,f1:0.152
epoch400:TRAIN -LOSS = 0.114,
 accuracy:0.098,precision:0.899,recall:0.084,f1:0.153
epoch500:TRAIN -LOSS = 0.115,
 accuracy:0.098,precision:0.969,recall:0.071,f1:0.132
epoch600:TRAIN -LOSS = 0.117,
 accuracy:0.098,precision:0.917,recall:0.082,f1:0.150
epoch700:TRAIN -LOSS = 0.112,
 accuracy:0.098,precision:0.919,recall:0.081,f1:0.148
epoch800:TRAIN -LOSS = 0.120,
 accuracy:0.098,precision:0.951,recall:0.077,f1:0.143
epoch900:TRAIN -LOSS = 0.108,
 accuracy:0.098,precision:0.825,recall:0.091,f1:0.163
epoch1000:TRAIN -LOSS = 0.111,
 accuracy:0.098,precision:0.940,recall:0.080,f1:0.148

 최종 테스트 :
 accuracy:0.098,precision:0.940,recall:0.080,f1:0.148


In [ ]:
binary_classification_exec(epoch_count=1000, report=100,mb_size = 10,adjust_ratio=True)

epoch100:TRAIN -LOSS = 0.332,
 accuracy:0.504,precision:0.973,recall:0.473,f1:0.636
epoch200:TRAIN -LOSS = 0.327,
 accuracy:0.504,precision:0.951,recall:0.493,f1:0.649
epoch300:TRAIN -LOSS = 0.315,
 accuracy:0.504,precision:0.913,recall:0.510,f1:0.654
epoch400:TRAIN -LOSS = 0.320,
 accuracy:0.504,precision:0.974,recall:0.484,f1:0.646
epoch500:TRAIN -LOSS = 0.312,
 accuracy:0.504,precision:0.949,recall:0.497,f1:0.652
epoch600:TRAIN -LOSS = 0.312,
 accuracy:0.504,precision:0.966,recall:0.490,f1:0.650
epoch700:TRAIN -LOSS = 0.314,
 accuracy:0.504,precision:0.981,recall:0.475,f1:0.640
epoch800:TRAIN -LOSS = 0.315,
 accuracy:0.504,precision:0.978,recall:0.476,f1:0.640
epoch900:TRAIN -LOSS = 0.288,
 accuracy:0.504,precision:0.885,recall:0.522,f1:0.657
epoch1000:TRAIN -LOSS = 0.299,
 accuracy:0.504,precision:0.911,recall:0.511,f1:0.655

 최종 테스트 :
 accuracy:0.504,precision:0.911,recall:0.511,f1:0.655


## 다중분류 softmax

In [ ]:
%run /content/drive/MyDrive/abalone.ipynb

[[ 0.      1.      0.     ...  0.101   0.15   15.    ]
 [ 0.      1.      0.     ...  0.0485  0.07    7.    ]
 [ 0.      0.      1.     ...  0.1415  0.21    9.    ]
 ...
 [ 0.      1.      0.     ...  0.2875  0.308   9.    ]
 [ 0.      0.      1.     ...  0.261   0.296  10.    ]
 [ 0.      1.      0.     ...  0.3765  0.495  12.    ]]
[['M', '0.455', '0.365', '0.095', '0.514', '0.2245', '0.101', '0.15', '15'], ['M', '0.35', '0.265', '0.09', '0.2255', '0.0995', '0.0485', '0.07', '7']]
총 데이터의 수(행): 4177
데이터의 70%의 미니배치 스텝 수: 29


In [ ]:
def multi_class_exec(epoch_count = 10, mb_size = 10, report = 1, train_rate = 0.75):
  multi_load_dataset()
  init_model()
  train_and_test(epoch_count,mb_size,report,train_rate)

In [ ]:
def multi_load_dataset():
  with open('/content/drive/MyDrive/faults.csv') as csvfile:
      csvreader = csv.reader(csvfile)
      next(csvreader, None)
      rows = []
      for row in csvreader:
          rows.append(row)
          
  global data, input_cnt, output_cnt
  input_cnt, output_cnt = 27, 7
  data = np.asarray(rows, dtype='float32')

In [ ]:
def fowrard_neuralnet(x):
  global weight,bias
  output=np.matmul(x,weight)+bias
  return output,x

In [ ]:
#소프트맥스 교차 엔트로피 연산 수식 필요
def forward_postproc(output,y):
  entropy=softmax_cross_entropy_with_logits(y,output)
  #여기서 신경망 연산에 대한 확률 분포가 필요
  loss=np.mean(entropy)
  return loss,[y,output,entropy]

In [ ]:
#오버플로우 발생하지 않기 위해 transpose 사용
def softmax(x):
  max_elem=np.max(x,axis=1)#열기준으로 추출
  diff=(x.transpose()-max_elem).transpose()#행렬 전환하여 수행 후 원상 복귀
  exp=np.exp(diff)
  sum_exp=np.sum(exp,axis=1)
  probs=(exp.transpose()/sum_exp).transpose()#분자와 분모의 나눗셈을 위해 행렬전환 후 연산. 그리고 다시 원상복귀를 위해 행렬전환
  return probs

In [ ]:
def softmax_cross_entropy_with_logits(labels,logits):
  #소프트맥스 교차 엔트로피 개선식의 신경망 예측에 대한 확률분포 𝑞 위치에 𝑙𝑜𝑔 연산의 폭주를 막기위
#한 아주 작은 양수값(𝜀)과 함께 더해주며 위치
  probs=softmax(logits)
  return -np.sum(labels*np.log(probs+1.0e-10),axis=1)

In [ ]:
def backprop_postproc(g_loss,aux_pp):
  y,output,entropy=aux_pp
  g_loss=1.0
  g_loss_entropy=1.0/np.prod(entropy.shape)
  g_entropy_output = softmax_cross_entropy_with_logits_derv(y,output)
  g_entropy=g_loss_entropy*g_loss
  g_output=g_entropy_output*g_entropy
  return g_output

In [ ]:
def softmax_cross_entropy_with_logits_derv(labels,logits):
  return softmax(logits)-labels

In [ ]:
def backprop_neuralnet(g_output,x):
  global weight,bias
  g_output_w=x.transpose()
  g_w=np.matmul(g_output_w,g_output)
  g_b=np.sum(g_output,axis=0)
  weight-=learning_rate*g_w
  bias-=learning_rate*g_b

In [ ]:
def run_test(test_x,test_y):
  output,_=forward_neuralnet(test_x)
  accuracy=eval_accuracy(output,test_y)
  return accuracy

In [ ]:
def eval_accuracy(output,y):
  estimate = np.argmax(output, axis=1)
  answer = np.argmax(y, axis=1)
  correct = np.equal(estimate, answer) 
  return np.mean(correct)

In [ ]:
multi_class_exec(epoch_count=1000,mb_size = 100, report=100, train_rate=0.7)

[[ 3.08973041e-03  2.51634909e-03 -3.28032758e-04 -2.66101920e-03
  -1.77570899e-03  2.95241149e-04  2.54071640e-03]
 [-4.23653950e-03 -4.78325574e-03  2.87569922e-03  2.20541285e-04
   8.82326713e-05 -2.14788553e-03  1.44523566e-04]
 [ 8.81042109e-04  3.74105978e-03 -1.47450325e-03  3.90400805e-04
  -5.43819761e-03  2.32888979e-04 -1.84819947e-03]
 [ 1.18390553e-03  2.93093534e-03  1.28245785e-03  2.05700692e-03
   2.71533619e-03  1.80297815e-03 -4.69820744e-03]
 [-1.01631371e-03 -8.11400146e-04 -2.61462313e-03 -5.32698561e-03
  -2.96594589e-03 -1.67706806e-03  2.24077137e-03]
 [-1.45120716e-03  1.12176430e-04  1.43491468e-04 -4.58285022e-03
   4.21244748e-03 -2.71984391e-04  2.13829856e-03]
 [ 3.10456306e-03  2.66816943e-03  6.77982720e-04  4.63651777e-04
   2.38280787e-03 -2.27282118e-03  2.44684733e-03]
 [ 2.22112257e-04 -4.02711575e-03 -1.81053719e-03 -6.17590440e-03
  -1.90413264e-03 -4.04689306e-03 -3.45918028e-03]
 [ 3.08317568e-03  2.42006334e-03  5.40368040e-03  3.58373212e-0

In [ ]:
learning_rate = 0.0001
multi_class_exec(epoch_count=1000,mb_size = 100, report=100, train_rate=0.7)

[[-2.62371302e-04  1.31659824e-03  5.94801933e-03  3.35475154e-03
  -3.30815562e-03  1.44012790e-03 -2.90217059e-03]
 [ 2.68147150e-03 -1.27163818e-03 -9.67848841e-04  2.27025214e-04
   2.74465093e-03 -2.93122933e-03 -1.45225502e-03]
 [-2.66927167e-03  1.71194937e-03  6.20534653e-04  2.16206828e-03
  -7.43288627e-04 -3.71464007e-03 -1.73295035e-03]
 [-1.48126062e-04  1.64900543e-03  1.97789763e-03 -1.00329836e-03
  -1.17504405e-03  2.77357485e-03  5.32141951e-04]
 [ 4.86368057e-05 -3.79626062e-03 -4.82488459e-03 -4.33569180e-03
  -3.66609283e-03  3.54099688e-03 -3.52794187e-03]
 [-1.54756446e-03  5.46558827e-03  2.86957601e-03 -2.95812536e-03
  -1.21932688e-03  1.64887274e-05 -4.83457663e-03]
 [ 5.30294975e-04  8.68804735e-04 -2.23142578e-03  1.06281843e-03
   1.33716439e-03 -1.32590453e-03  2.97843537e-03]
 [ 8.65351197e-04  1.04355785e-03  1.85717949e-03  1.00565422e-03
   6.56661628e-03  4.10350105e-03  4.86816020e-04]
 [-2.50665696e-03  1.52233111e-03 -5.15402953e-03  2.20766578e-0